In [ ]:
from pathlib import Path
from math import floor
import numpy as np
from hartufo import CipicPlane, AriPlane, ListenPlane, CrossModPlane, BiLiPlane, ItaPlane, HutubsPlane, RiecPlane, Sadie2Plane, Princeton3D3APlane, ChedarPlane, WidespreadPlane, ScutPlane, SonicomPlane
from hartufo import HrirSpec
from hartufo.full import split_by_angles
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt

Inspired by Andreopoulou and Roginska's paper "Towards the Creation of a Standardized HRTF Repository", presented at the 131st convention of the AES (2011).

### Configuration

In [ ]:
base_dir = Path('../hartufo-collections')

In [ ]:
plane_collections = (
    # (CipicPlane, base_dir / 'CIPIC'),
    (ListenPlane, base_dir / 'Ircam Listen'),
    (CrossModPlane, base_dir / 'Ircam CrossMod'),
    # (BiLiPlane, base_dir / 'Ircam BiLi'),
    (AriPlane, base_dir / 'ARI'),
    (RiecPlane, base_dir / 'RIEC'),
    (ItaPlane, base_dir / 'ITA'),
    # (Princeton3D3APlane, base_dir / '3D3A'),
    # (Sadie2Plane, base_dir / 'SADIE II'),
    (ScutPlane, base_dir / 'SCUT'),
    (HutubsPlane, base_dir / 'HUTUBS'),
    # (ChedarPlane, base_dir / 'CHEDAR'),
    # (WidespreadPlane, base_dir / 'Widespread'),
    # (SonicomPlane, base_dir / 'SONICOM'),
)

In [ ]:
plane = 'horizontal'
domain = 'magnitude_db'
side = 'both-left'

### Determine common ground

In [ ]:
# Get positions, samplerates and lengths for all datasets
plane_angles = []
samplerates = []
hrir_lengths = []
for collection, data_dir in plane_collections:
    plane_offset = -0.72 if collection == ItaPlane and plane == 'horizontal' else 0
    ds = collection(data_dir, plane, domain='time', side='left', subject_ids='first', plane_offset=plane_offset, verify=False)
    plane_angles.append(set(ds.plane_angles))
    samplerates.append(ds.hrir_samplerate)
    hrir_lengths.append(ds.hrir_length)
# Determine common angles and minimum samplerate and length
common_plane_angles = sorted(set.intersection(*plane_angles))
samplerate = min(samplerates)
hrir_length = min([floor(samplerate / sr * l) for sr, l in zip(samplerates, hrir_lengths)])

In [ ]:
common_plane_angles, samplerate, hrir_length

### Read complete collection

In [ ]:
# Read all datapoints with common parameters
datasets = []
for collection, data_dir in plane_collections:
    print(data_dir)
    plane_offset = -0.72 if collection == ItaPlane and plane == 'horizontal' else 0
    exclude_ids = (1, 2) if collection == Sadie2Plane else None
    ds = collection(
        data_dir, plane, domain, side, plane_angles=common_plane_angles,
        plane_offset=plane_offset, hrir_samplerate=samplerate, hrir_length=hrir_length, hrir_min_phase=True, exclude_ids=exclude_ids, verify=False,
    )
    datasets.append(ds)

### Select balanced subset of data

In [ ]:
individual_measurements = False

In [ ]:
max_collection_size = min(len(d) for d in datasets)
# max_collection_size = 999
if individual_measurements: # each datapoint is a single measurement location (HRIR)
    hrtf_size = datasets[0][0]['features'].shape[-1]
    balanced_features = [np.array([angle_ds[:max_collection_size]['features'] for angle_ds in split_by_angles(ds)]).reshape(-1, hrtf_size) for ds in datasets]
else: # each datapoint is all (retained) measurement locations (HRIRs) for a subject/side combination
    balanced_features = [np.array([ex.reshape(-1) for ex in ds[:max_collection_size]['features']]) for ds in datasets]

In [ ]:
len(balanced_features), balanced_features[0].shape

### Standard scaling per collection

In [ ]:
means = [np.mean(f, axis=0) for f in balanced_features]
stdevs = [np.std(f, axis=0) for f in balanced_features]

In [ ]:
len(means), means[0].shape

In [ ]:
normalised_features = [(f - mean) / stdev for f, mean, stdev in zip(balanced_features, means, stdevs)]

In [ ]:
len(normalised_features), normalised_features[0].shape, normalised_features[5].shape

### Calculate distance between datapoints

In [ ]:
prenorm_distances = pairwise_distances(np.row_stack(balanced_features), metric='euclidean')
postnorm_distances = pairwise_distances(np.row_stack(normalised_features), metric='euclidean')

### Plot distances

In [ ]:
plt.matshow(prenorm_distances)
plt.matshow(postnorm_distances)
plt.show()